### Ingest qualifying folder

##### Read the multiple json file with muliple lines from the folder using spark df reader

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, FloatType, TimestampType
from pyspark.sql.functions import col, current_timestamp, lit, to_timestamp,concat

In [0]:
qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                        StructField("raceId", IntegerType(), True),
                                        StructField("driverId", IntegerType(), True),
                                        StructField("constructorId", IntegerType(), True),
                                        StructField("number", IntegerType(), True),
                                        StructField("position", IntegerType(), True),
                                        StructField("q1", StringType(), True),
                                        StructField("q2", StringType(), True),
                                        StructField("q3", StringType(), True)])

In [0]:
qualifying_df = spark.read \
.schema(qualifying_schema) \
.option("multiline", True) \
.json(f"{raw_folder_path}/qualifying")

#another way = .csv("dbfs:/mnt/formula1dlstudy/raw/lap_times/lap_times_split*.csv")

##### Select the columns

In [0]:
qualifying_final_df = qualifying_df.withColumnRenamed("qualifyId", "qualify_id") \
                                      .withColumnRenamed("raceId", "race_id") \
                                      .withColumnRenamed("driverId", "driver_id") \
                                      .withColumnRenamed("constructorId", "constructor_id") \
                                     .withColumn("ingestion_date", current_timestamp()) \
                                       .withColumn("datasource", lit(v_data_source))

Write the result as parquet file

In [0]:
#qualifying_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/qualifying")

In [0]:
qualifying_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.qualifying")

In [0]:
#%fs
#ls /mnt/formula1dlstudy/processed/qualifying

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/qualifying"))

In [0]:
%sql
SELECT * FROM f1_processed.qualifying

In [0]:
dbutils.notebook.exit("Success")